<a href="https://colab.research.google.com/github/calvingehrer/automaticHintGeneration/blob/main/hintGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Call Functions


**Call hint function for all questions where answer is Location**

In [47]:
for index, row in location_df.iterrows():
  location = row["Answer"]
  questionLocation = row["Question"]
  with open("hintsLocation.txt", 'a') as writefile:
    writefile.write("\n\n" + questionLocation + " (" + location + ")\n\n")
    writefile.close()


  #get Wikidata ID for SPARQL-Query
  locationQitem = getQitemOfName(location)

  #execute SPARQL-Query
  queryResults_df = allHintsLocation(locationQitem)

  #getFinalHints
  listHintsLocation = getFormatedHintsLocation(queryResults_df)    

  #write hints in file in random order
  writeLocationHintsInFile(listHintsLocation)

**Call hint function for all questions where answer is Person**



In [48]:

for index, row in person_df.iterrows():
  person = row["Answer"]
  questionPerson = row["Question"]

  with open("hintsPerson.txt", 'a') as writefile:
    writefile.write("\n\n" + questionPerson + " ("+ person +")\n\n")
    writefile.close()
  
  #get Wikipedia ID for SPARQL-Query
  personQitem = getQitemOfName(person)

  #execute SPARQL-Query
  queryResults_df = allHintsPerson(personQitem)
  
  #get Final Hints
  listHintsPerson = getFormatedHintsPerson(queryResults_df)

  #write hints in file in random order
  writePersonHintsInFile(listHintsPerson)


**Call hint function for all question where answer is Year**

In [ ]:
for index, row in year_df.iterrows():
  year = row["Answer"]
  questionYear = row["Question"]
  hintsYear(row["Answer"])

# Load File

In [46]:
import pandas as pd

df = pd.ExcelFile("./testSet.xlsx").parse("Sheet1")
x = []
x.append(df["Answer"])

dataPerson = []
dataYear = []
dataLocation = []

for index, row in df.iterrows():
  if(row["Category"] == "Person"):
    dataPerson.append([row["Question"], row["Answer"]])
  elif(row["Category"] == "Year"):
    dataYear.append([row["Question"], row["Answer"]])
  elif(row["Category"] == "Location"):
    dataLocation.append([row["Question"], row["Answer"]])

person_df = pd.DataFrame(dataPerson, columns=["Question", "Answer"])
year_df = pd.DataFrame(dataYear, columns=["Question", "Answer"])
location_df = pd.DataFrame(dataLocation, columns=["Question", "Answer"])

# All imports

In [4]:
!pip install sparqlwrapper

     |████████████████████████████████| 482 kB 6.8 MB/s 
     |████████████████████████████████| 41 kB 630 kB/s 


In [5]:
!pip install git+https://github.com/Commonists/pageview-api.git

  Cloning https://github.com/Commonists/pageview-api.git to /tmp/pip-req-build-tek97aej
  Running command git clone -q https://github.com/Commonists/pageview-api.git /tmp/pip-req-build-tek97aej
  Created wheel for pageviewapi: filename=pageviewapi-Undefined-py3-none-any.whl size=5309 sha256=bd7b661e69fbf65591fb7227d1ab0901ea68b14aaa0c3b8db63e193a7aaf9697
  Stored in directory: /tmp/pip-ephem-wheel-cache-hgqj57yw/wheels/69/f3/cc/8824053978e3f964d9bd14b8fc89d9908857e966279d625915
Failed to build pageviewapi
    Running setup.py install for pageviewapi ... done
  DEPRECATION: pageviewapi was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [6]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 4.0 MB 28.2 MB/s 
     |████████████████████████████████| 1.2 MB 39.5 MB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 
     |████████████████████████████████| 6.5 MB 33.3 MB/s 
     |████████████████████████████████| 895 kB 50.7 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=f37f3aa227b523213f7bae08ec17a64da50b6a2c7bf61844ab73096f87ed187d
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=21621767cc9e1a5aa958dc9d0d610a1b98adc31821dea7564909a7efa53e2d71
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [8]:
import pandas as pd
pd.set_option('display.max_colwidth',1000)

In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")


In [10]:
import requests

In [11]:
import bs4
from bs4 import BeautifulSoup, NavigableString, Tag

In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
import wikipedia

In [14]:
import random

In [15]:
import pageviewapi

In [16]:
import spacy

# Functions for reuse

In [17]:
#get Wikidata ID
def getQitemOfName(entity):
  
  searched = "'"+entity+"'"
  sparql.setQuery('''
  SELECT ?item ?itemLabel WHERE {
    ?item rdfs:label '''+searched+'''@en.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }

  }
  ''')
  
  sparql.setReturnFormat(JSON)
  entities = sparql.query().convert()
  entities_df = pd.json_normalize(entities['results']['bindings'])
  words = entities_df[["item.value"]].iloc[0].str.split("/")[0]
  return words[-1]

In [18]:
#check if entity is a person
def checkIfHuman(entity):

  listHumans = []

  searched = ""

  for item in entity:
    searched += "'"+item+"' "

  #print(searched)
  
  try:

    sparql.setQuery('''
        SELECT distinct ?item ?itemLabel 
        WHERE{
          VALUES ?entity{'''+searched+'''}
          ?item ?label ?entity.  
          ?item wdt:P31 wd:Q5 .
          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.	
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
        }
    ''')
      
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])

    #print(entities_df)

    for index, row in entities_df.iterrows():
      listHumans.append(row["itemLabel.value"])

    return listHumans
    
  except:
    return None

In [19]:
#get name of entity from Wikidata ID

def getLabelOfQitems(entities):

  finalLabels = []
  for item in entities:
    #searched = "'"+item+"'"
    sparql.setQuery('''
    SELECT * WHERE {
        wd:'''+item+ ''' rdfs:label ?label .
        FILTER (langMatches( lang(?label), "EN" ) )
    } LIMIT 1
    ''')
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])
    label = entities_df["label.value"].iloc[0]
    finalLabels.append(label)

  return finalLabels  

In [20]:
#function to get all backlinks to wikipage and saves backlinks >= 1000 in array
#source: https://www.mediawiki.org/wiki/API:Backlinks#Response

import requests

def getBacklinks(entities):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  listWithBacklink500 = []
  dictWithAllEntries = {}

  for val in entities:
    #print(row["itemLabel.value"])
    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": val,
        "bllimit": "max"
    }

    try:
      R = S.get(url=URL, params=PARAMS)
      DATA = R.json()
   

      BACKLINKS = DATA["query"]["backlinks"]

      count = 0
      for links in BACKLINKS:
        count +=1

      
      while "continue" in DATA:
        if (count >= 1000):
          break;
        blcontinue = DATA["continue"]["blcontinue"]
        PARAMS["blcontinue"] = blcontinue

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        BACKLINKS = DATA["query"]["backlinks"]

        for links in BACKLINKS:
          count += 1
      
      dictWithAllEntries[val] = count
      #print(val + ": " + str(count))

    except ValueError:
      print("Value Error")

    if (count >= 1000):
      listWithBacklink500.append(val) 

  if (len(listWithBacklink500) > 0):
    #return getPageViews(listWithBacklink500)
    return listWithBacklink500

  else:
    listMostBacklinks = []
    sortedDict = dict(sorted(dictWithAllEntries.items(), key=lambda item:item[1]))

    for k, v in sortedDict.items():
      listMostBacklinks.append(k)
    
    numberToPass = len(listMostBacklinks)//3
        
    return listMostBacklinks[-numberToPass:]

In [21]:
#function to get all backlinks to wikipage and saves backlink == 500 in array
#source: https://www.mediawiki.org/wiki/API:Backlinks#Response

import requests

def getBacklinksDict(entities):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  dictWithBacklink500 = {}
  dictWithAllEntries = {}

  for key, val in entities.items():

    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": key,
        "bllimit": "max"
    }

    try:
      R = S.get(url=URL, params=PARAMS)
      DATA = R.json()
   

      BACKLINKS = DATA["query"]["backlinks"]

      count = 0
      for links in BACKLINKS:
        count +=1

      
      while "continue" in DATA:
        if (count >= 1000):
          break;
        blcontinue = DATA["continue"]["blcontinue"]
        PARAMS["blcontinue"] = blcontinue

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        BACKLINKS = DATA["query"]["backlinks"]

        for links in BACKLINKS:
          count += 1
      
      dictWithAllEntries[key] = count
      #print(val + ": " + str(count))

    except ValueError:
      print("Value Error")
    except KeyError:
      print("Key Error")

    if (count >= 1000):
      dictWithBacklink500.update({key: val}) 

  if (len(dictWithBacklink500) > 0):
    #return getPageViews(listWithBacklink500)
    return dictWithBacklink500

    #print(listMostBacklinks[-numberToPass:])

    
    return listMostBacklinks[-numberToPass:]

In [22]:
#get avg pageViews of entities

def getPageViewsList(listEntities):
  avgDict = {}
  
  for val in listEntities:
    try:
      resp = pageviewapi.per_article('en.wikipedia', val, '20191106', '20201120', access='all-access', agent='all-agents', granularity='monthly')
    #print(resp)
      avgViews = 0
      count = 0
      for i in resp.get("items"):
        #print(i.get("views"))
        avgViews += i.get("views")
        count += 1
      avgDict.update({val: avgViews//count})
    except:
      print("Page not found to check pageViews for page: " + val)
      
  #return (max(avgDict, key=avgDict.get))
  return avgDict


  


In [23]:
#get avg pageViews of entity
import pageviewapi


def getPageViews(entity):
  avgDict = {}
  
  try:
    resp = pageviewapi.per_article('en.wikipedia', entity, '20191106', '20201120', access='all-access', agent='all-agents', granularity='monthly')

    avgViews = 0
    count = 0
    for i in resp.get("items"):
      #print(i.get("views"))
      avgViews += i.get("views")
      count += 1

    pageViews = avgViews//count
    #avgDict.update({val: avgViews//count})

    #return (max(avgDict, key=avgDict.get))
    return pageViews
  #print(resp)
  except:
    print("Page not found to check pageViews for page: " + entity)
    return -1
    


**Check if entity is human**

In [24]:
#check if entity is a person
#return list of persons

def checkIfHuman10(entity):

  #listHumans = []

  #for test in entity:
    #print(item)
  searched = "'"+entity+"'"

  #print(searched)

  try:

    sparql.setQuery('''
        SELECT distinct ?item ?itemLabel 
        WHERE{
          ?item ?label '''+searched+'''.  
          ?item wdt:P31 wd:Q5.
          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.	
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
        }
    ''')
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])

    #print(entities)
    #print(entities_df["itemLabel.value"].item())

    #for index, row in entities_df.iterrows():
    return entities_df["itemLabel.value"].item()
    
    
  except:
    return None


In [25]:
#check if entity is a person
#return list of persons

def checkIfHuman5(entity):

  listHumans = []

  for test in entity:
    #print(item)
    searched = "'"+test+"'"

  #print(searched)

    try:

      sparql.setQuery('''
          SELECT distinct ?item ?itemLabel 
          WHERE{
            ?item ?label '''+searched+'''.  
            ?item wdt:P31 wd:Q5.
            ?article schema:about ?item .
            ?article schema:inLanguage "en" .
            ?article schema:isPartOf <https://en.wikipedia.org/>.	
            SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
          }
      ''')

        
      sparql.setReturnFormat(JSON)
      entities = sparql.query().convert()
      entities_df = pd.json_normalize(entities['results']['bindings'])

      #print(entities)
      #print(entities_df["itemLabel.value"].item())

      #for index, row in entities_df.iterrows():
      listHumans.append(entities_df["itemLabel.value"].item())
      
      
    except:
      print(None)

  return listHumans

In [26]:
#check if entity is a person
#return list of persons

def checkIfHuman4(entity):

  listHumans = []

  searched = ""

  for item in entity:
    #print(item)
    searched += "'"+item+"' "

  #print(searched)

  try:

    sparql.setQuery('''
        SELECT distinct ?item ?itemLabel 
        WHERE{
          VALUES ?entity{'''+searched+'''}
          ?item ?label ?entity.  
          ?item wdt:P31 wd:Q5.
          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.	
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
        }
    ''')
      
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])

    #print(entities_df)

    for index, row in entities_df.iterrows():
      listHumans.append(row["itemLabel.value"])

    return listHumans
    
  except:
    return None

In [27]:
def checkIfHuman2(entitiesDict):  

  listIsHuman = []
  stringEntities = ""

  for key in entitiesDict:

    stringEntities += "'"+key+"' "

  sparql.setQuery('''
  SELECT ?item ?itemLabel ?entity ?entityLabel
  WHERE {
    VALUES ?entity{'''+stringEntities+'''}
    ?entity wdt:P31 ?item
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  }
  ''')
  sparql.setReturnFormat(JSON)
  isHuman = sparql.query().convert()
  isHuman_df = pd.json_normalize(isHuman['results']['bindings'])
  #print(isHuman_df["itemLabel.value"].item() == "human")
  try:
    if (isHuman_df["itemLabel.value"].item() == "human"):
      listIsHuman.append(val)
  except Exception:
    print("No human (dont know exactly why this exception is thrown)")
  for index, row in isHuman_df.iterrows():
    if(row["item.value"] == "http://www.wikidata.org/entity/Q5"):
      listIsHuman.append(row["entityLabel.value"])
    

  return listIsHuman
  


In [28]:
#if parantheses in entity -> remove

def removeParanthesesDict(inputDict):

  newDict = {}
  for key, val in inputDict.items():
    if ('(' in key):
      newElem = re.sub("[\(\[].*?[\)\]]", "", key).rstrip()
      newDict.update({newElem: val})
    else:
      newDict.update({key: val})

  return newDict

#Hints if answer is a Location

In [29]:
#Query with all properties to get data

def allHintsLocation(entity):
  sparql.setQuery('''
  SELECT ?itemLabel ?property
  WHERE
  {  
    VALUES ?property {wdt:P35 wdt:P30 wdt:P36 wdt:P610 wdt:P1082 wdt:P421 wdt:P38 wdt:P17 wdt:P1376 wdt:P131 wdt:P6 wdt:P1830 wdt:P47 wdt:P206 wdt:P37 wdt:P463 wdt:793}
    wd:'''+entity+''' ?property ?item.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
  ''')
  sparql.setReturnFormat(JSON)
  hintsLocation = sparql.query().convert()

  hintsLocation_df = pd.json_normalize(hintsLocation['results']['bindings'])

  return hintsLocation_df

In [30]:
#create template based hints

def getFormatedHintsLocation(queryResults_df):
  timeZone = []
  isLocatedIn = []
  ownerOf = []
  shareBorder = []
  bodyOfWater = []
  languages = []
  memberOf = []
  significantEvents = []

  listHintsLocation = []

  searchString = "http://www.wikidata.org/prop/direct/"
  
  for index, row in queryResults_df.iterrows():
    if(not location in row["itemLabel.value"]):
      if(row["property.value"] == searchString + "P35"):
        listHintsLocation.append("Head of state of searched country is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P30"):
        listHintsLocation.append("The searched location is on continent " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P36"):
        listHintsLocation.append("The capital of searched country is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + ""):
        listHintsLocation.append("The highest point in searched location is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P1082"):
        listHintsLocation.append("The searched location has a population of " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P38"):
        listHintsLocation.append("Currency in searched location is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P17"):
        if(not location in row["itemLabel.value"]):
          listHintsLocation.append("The searched location is in country "+ row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P1376"):
        listHintsLocation.append("The searched city is capital of " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P131"):
        listHintsLocation.append("The searched location is located in " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P6"):
        listHintsLocation.append("Head of government of searched location is " + row["itemLabel.value"])
      
      #if get multiple values save all in list and pick one afterwards
      elif(row["property.value"] == searchString + "P1830"):
        if(not location in row["itemLabel.value"]):
          ownerOf.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P47"):
        shareBorder.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P206"):
        bodyOfWater.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P37"):
        if(not location in row["itemLabel.value"]):
          languages.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P421"):
        timeZone.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P463"):
        memberOf.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P793"):
        if(not location in row["itemLabel.value"]):
          significantEvents.append(row["itemLabel.value"])

  #getting more than one result with one query, pick one for output
  if(len(ownerOf) > 0):
    listHintsLocation.append("The searched location is owner of " + ownerOf[0])
  if(len(shareBorder) > 0):
    listHintsLocation.append("The searched location shares border with " + shareBorder[0])
  if(len(bodyOfWater) > 0):
    listHintsLocation.append("The next body of water of searched location is " + bodyOfWater[0])
  if(len(memberOf) > 0):
    listBacklinks = getBacklinks(memberOf)
    dictPageViews = getPageViewsList(listBacklinks)
    listHintsLocation.append("Searched location is member of " + (max(dictPageViews, key=dictPageViews.get)))
  if(len(significantEvents) > 0):
    listBacklinks = getBacklinks(significantEvents)
    dictPageViews = getPageViewsList(listBacklinks)
    listHintsLocation.append("A significant event happened in this location was " + (max(dictPageViews, key=dictPageViews.get)))

  #if getting more than one language concatenate all
  commaCounter = len(languages)-1
  stringLanguages = ""
  for language in languages:
    if(commaCounter > 0):
      stringLanguages += language + ", "
      commaCounter -= 1
    else:
      stringLanguages += language
  if(stringLanguages != ""):
    listHintsLocation.append("Spoken language(s) in searched location is/are " + stringLanguages)
  if(len(timeZone) > 0):
    listHintsLocation.append("The searched location is in time Zone " + timeZone[0])

  
  return listHintsLocation


In [31]:
def writeLocationHintsInFile(listHintsLocation):
  randomHintsList = random.sample(listHintsLocation, len(listHintsLocation))
  
  for item in randomHintsList:
    with open("hintsLocation.txt", 'a') as writefile:
      writefile.write(item + "\n")
      writefile.close()

#Hints if answer is a Person

In [32]:
#Query with all properties to get data

def allHintsPerson(entity):
  sparql.setQuery('''
  SELECT ?itemLabel ?property
  WHERE
  {  
    VALUES ?property {wdt:P21 wdt:P27 wdt:P569 wdt:P19 wdt:P1971 wdt:P106 wdt:P1340 wdt:P1884 wdt:P2048 wdt:P39 wdt:P69 wdt:P512 wdt:P102 wdt:P3602 wdt:P800 wdt:P166 wdt:P3373 wdt:P1412 wdt:P413 wdt:P118 wdt:P54}
    wd:'''+entity+''' ?property ?item.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
  ''')
  sparql.setReturnFormat(JSON)
  hintsPerson = sparql.query().convert()

  hintsPerson_df = pd.json_normalize(hintsPerson['results']['bindings'])
  
  return hintsPerson_df

In [33]:
#create temolate based hints

def getFormatedHintsPerson(queryResults_df):

  listOccupation = ["Searched person was occuupied as "]
  listPositionHeld = ["Searched person held the position(s) "]
  listEducatedAt = ["Searched person was educated at "]
  listAcademicDegrees = ["Searched person has academic degrees "]
  listPoliticParty = ["Searched was/is member of politic party "]
  listCandidacyElection = ["Searched person was candidacy at election(s) "]
  listNotableWork = ["Searched person did notable work "]
  listAwardsReceived = ["Searched person received awards "]
  listLanguages = ["Searched person speaks language(s) "]
  listPlayedPositions = ["Searched person played position "]
  listLeague = ["Searched person played in leagues "]
  listTeams = ["Searched person played in teams "]
  listCitizenship = ["Searched person has following citizenships "]
  listSiblings = []
  listHair = []

  listHintsPerson = []

  for index, row in queryResults_df.iterrows():
    if(row["property.value"] == "http://www.wikidata.org/prop/direct/P21"):
      listHintsPerson.append("Gender of searched person is " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P569"):
      listHintsPerson.append("Searched person was born in " + row["itemLabel.value"][:4])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P19"):
      listHintsPerson.append("Searched person was born in " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1971"):
      listHintsPerson.append("Number of children of seared person is " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1340"):
      listHintsPerson.append("Searched person has " + row["itemLabel.value"] + " eyes")    
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P2048"):
      listHintsPerson.append("Searched persons height is " + row["itemLabel.value"])    

    
    #if get multiple values save all in list and pick one afterwards
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P27"):
      listCitizenship.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P106"):
      listOccupation.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P39"):
      listPositionHeld.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P69"):
      listEducatedAt.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P512"):
      listAcademicDegrees.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P102"):
      listPoliticParty.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P3602"):
      listCandidacyElection.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P800"):
      listNotableWork.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P166"):
      listAwardsReceived.append(row["itemLabel.value"])

    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1412"):
      listLanguages.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P413"):
      listPlayedPositions.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P118"):
      listLeague.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P54"):
      listTeams.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P3373"):
      listSiblings.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1884"):
      listHair.append(row["itemLabel.value"])

  #if too many results could be return check
  #if more thane 3 -> cut
  listResultsCut = [listPositionHeld, listAcademicDegrees, listPoliticParty, 
                 listCandidacyElection, listNotableWork, listLanguages, 
                 listPlayedPositions, listLeague, listTeams, listCitizenship, listAwardsReceived, listEducatedAt, listOccupation]
  for singleList in listResultsCut:
    cutResults(singleList, listHintsPerson)

  #just get one hair color
  if(len(listHair) > 0):
    listHintsPerson.append("The searched person has " + listHair[0])

  #get number of siblings
  if(len(listSiblings) > 0):
    listHintsPerson.append("The searched person has " + str(len(listSiblings)) + " siblings")  
  
  return listHintsPerson

In [34]:
#if more results than 3 -> print 3 results + "and x others"

def cutResults(listResults, listHintsPerson):
  
  numResults = len(listResults)-1
  resultsLeft = numResults-3

  if (len(listResults) > 1):
    if(resultsLeft > 0):
      listHintsPerson.append(getConcatenationMultipleResults(listResults[0:4])+ " and " + str(resultsLeft) + " others")
    else:
      listHintsPerson.append(getConcatenationMultipleResults(listResults))

In [35]:
#concatenate multiple results with comma

def getConcatenationMultipleResults(resultList):
  concatenation = ""
  commaCounter = len(resultList)-1
  for item in resultList:
    if(commaCounter == len(resultList)-1):
      concatenation += item
      commaCounter-=1
    elif(commaCounter > 0):
      concatenation += item + ", "
      commaCounter-=1
    elif(commaCounter == 0):
      concatenation += item
  
  return concatenation


In [36]:
def writePersonHintsInFile(listHintsPerson):
  with open("hintsPerson.txt", 'a') as writefile:
    #writefile.write("\n\n---Hints for Person " + person + "---\n\n")
    randomHintsList = random.sample(listHintsPerson, len(listHintsPerson))
    for item in randomHintsList:
      writefile.write(item + "\n")
    writefile.close()

# Hints if answer is a Year

**Get all outlinks of wikipedia year page and pass to getBacklinks. Than look for return values on page.content and print line**

In [50]:
def hintsYear(year):

  page = wikipedia.page(year, auto_suggest=False)
  pageContent = page.content

  #get all links from year page
  dictPageLinks = getOutlinks2(year)

  #get rid of unnesercery links
  filteredPageLinks = filterDict(dictPageLinks)

  
  #get backlinks of all links
  dictBacklink = getBacklinksDict(filteredPageLinks)
  #first200 = {k: filteredPageLinks[k] for k in list(filteredPageLinks)[:200]}
  #dictBacklink = getBacklinksDict(first200)

  #remove parantheses of links
  dictAdjusted = removeParanthesesDict(dictBacklink)

  #get all possible hint (all lines with a entity #backlinks > 1000)
  dictPossibleHints = findLineInTextEvent(dictAdjusted, year)

  with open('./resultsEvents.txt', 'a') as writefile:
    writefile.write("Question: " + questionYear + "("+ str(year) + ")---\n")

  #check if entity because of which line was found is subject
  #if it is not the subject -> drop
  dictFinalHints = getFinalHints(dictPossibleHints)

  #get rid of number of foundevents at end of line
  #need to put number at the and because it is dict and would be overwritten
  dictFinalHintsAdjusted = {}
  for key, val in dictFinalHints.items():
    dictFinalHintsAdjusted.update({key.split('///', 1)[0]: val})  

  #returns dict sorted after PageViews
  dictSortedPV = sortHintsNumberPVSubject(dictFinalHintsAdjusted)

  #get highest PageView
  maxPV = list(dictSortedPV.values())[0]

  #sort hints PageViews and Simscore combined
  sortHintsUtilityScore(questionYear, dictSortedPV, maxPV)

  #listHumansNames = checkIfHuman5(dictAdjusted.keys())
  #print(listHumansNames)

  listHumansNames = []

  for key, val in dictAdjusted.items():
    result = checkIfHuman10(key)
    if(result != None):
      listHumansNames.append(result)


  #if(not listHumansNames == None):
    #listHumansNames = getLabelOfQitems(listHumans)

    #dictHumansPV = getPageViewsList(listHumansNames)

    #sortedDictHumansPV = dict(sorted(dictHumansPV.items(), key=lambda item:item[1]))

    #findLineInTextBirthDeath(sortedDictHumansPV.keys(), year)

  findLineInTextBirthDeath(listHumansNames, year)

**Function to find line of most important link in year content**

In [38]:
import re

def findLineInTextEvent(entitiesEvent, year):

  page = wikipedia.page(year, auto_suggest=False)
  pageText = page.content
  splitText = []

  splitText = (re.split("== Events ==|== Births ==", pageText ))

  contentEvents = splitText[1]

  foundEvent = 0

  dictHintsEntity = {}

  for key, val in entitiesEvent.items():
    #if (foundEvent == 20):
      #break;

    for line in contentEvents.split("\n"):
      #if (foundEvent == 20):
        #break;

      if val in line:
        words = line.split()
        #counterEntitiesInLine = 0
        if(len(words) < 15):
          dictHintsEntity.update({line + "///" + str(foundEvent): val})
          #listHints.append(line)
          foundEvent += 1
          #break;
  return dictHintsEntity

In [39]:
def findLineInTextBirthDeath(entitiesHuman, year):

  page = wikipedia.page(year, auto_suggest=False)
  pageText = page.content
  splitText = []

  splitText = (re.split("== Births ==|== Deaths ==", pageText ))


  contentBirths = splitText[1]
  contentDeaths = splitText[2]

  foundBirth = 0
  foundDeath = 0

  #open file and append results
  with open('./resultsBirthDeath.txt', 'a') as writefile:
    writefile.write("---Birth/Death in year " + str(year) + "---")

    for entity in entitiesHuman:
      #if (foundBirth == 3 and foundDeath == 3):
        #writefile.write("\n")
        #break;

      for line in contentBirths.split("\n"):
        #if (foundBirth == 3):
          #break;

        if entity in line:
          writefile.write("Found with entity " + entity + "\n")
          writefile.write("Birth: " + line + "\n")
          writefile.write("\n")
          foundBirth +=1
          break;

      for line in contentDeaths.split("\n"):
        #if (foundDeath == 3):
          #break;
        if entity in line:
          writefile.write("Found with entity " + entity + "\n")
          writefile.write("Death: " + line + "\n")
          writefile.write("\n")
          foundDeath += 1
          break;
    writefile.write("\n")
      

In [40]:
def sortHintsNumberPVSubject(dictHints):

  with open('./resultsEvents.txt', 'a') as writefile: 

    dictFinalHintsPV = {}

    for key, val in dictHints.items():
      dictFinalHintsPV.update({key: getPageViews(val)})

    dictFinalHintsSortedPV = dict(sorted(dictFinalHintsPV.items(), key=lambda item: item[1], reverse=True))
    """writefile.write("\n--Final Hints sorted #PV---\n")
    for key, val in dictFinalHintsSortedPV.items():
      writefile.write(key + "(" + str(val) + " PageViews)\n")"""

  return dictFinalHintsSortedPV



In [41]:


def sortHintsUtilityScore(question, dictFinalHintsSortedPV, maxPV):
  model = SentenceTransformer('bert-base-nli-mean-tokens')

  question_embedding = model.encode(question)

  alpha = 0.5

  #sentence_embeddings = model.encode(hints)

  dictHintsBERT = {}

  for key, val in dictFinalHintsSortedPV.items():
    sentence_embedding = model.encode(key)
    simScore = cosine_similarity([question_embedding], [sentence_embedding]).item()
    impScore = alpha * (val/maxPV) + (1-alpha) * simScore
    dictHintsBERT.update({key: impScore})

  with open('./resultsEvents.txt', 'a') as writefile:

    dictHintsBERT = dict(sorted(dictHintsBERT.items(), key=lambda item: item[1], reverse=True))
    writefile.write("\n--Final Hints sorted Utility Score---\n")
    for key, val in dictHintsBERT.items():
      writefile.write(key + "(" + str(val) + " Utility Score)\n")

  return dictHintsBERT
 

In [42]:
def get_subject_phrase(doc):
    for token in doc:
        if ("subj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [43]:
def getFinalHints(dictPossibleHints):

  nlp = spacy.load("en")
  dictFinalHintsSubject = {}
  
  for key, val in dictPossibleHints.items():
    doc = nlp(key)
    subject = get_subject_phrase(doc)
    if (val in str(subject)):
      dictFinalHintsSubject.update({key: val})
      #listFinalHints.append(key)

  return dictFinalHintsSubject

In [44]:

def getOutlinks2(year):

  URL = 'https://en.wikipedia.org/wiki/' + str(year)

  # Fetch all the HTML source from the url
  response = requests.get(URL)


  soup = bs4.BeautifulSoup(response.text, 'html.parser')

  listTextOfLinks = []
  listLinks = []

  dictLinkName = {}


  for link in soup.find_all("a"):
    listTextOfLinks.append(link.get_text())
    dictLinkName.update({str(link.get("href")).split('/')[-1].replace('_', ' '): link.get_text()})

  #newDict = filterDict(dictLinkName)

  #print(len(dictLinkName))
  #print(len(newDict))
  #print(newDict)

  return dictLinkName

In [45]:
def filterDict(dictNameLink):

  patternYear = r"[1-3][0-9]{3}"
  patternCentury = r"[1-2][0-9]\w+.century"
  patternAd = r"AD.\d\d\d\d"
  patternMonth = r"[a-zA-Z]+\s\d+"
  patternMillennium = r".+millennium"
  patternHashtag = r"#"
  patternCalendar = r"calendar"
  patternCategory = r"category"
  patternList = r"list"
  patternTemplate = r"template"
  patternWikipedia = r"wikipedia"
  patternIndex = r"index"
  patternHtml = r"html"
  patternPercent = r"%"

  newDict = {}

  for key, val in dictNameLink.items():
    if(not re.match(patternYear, key) and
       not re.match(patternCentury, key) and 
       not re.match(patternAd, key) and 
       not re.match(patternMonth, key) and
       not re.search(patternHashtag, key) and
       not re.search(patternCalendar, key, re.IGNORECASE) and
       not re.search(patternCategory, key, re.IGNORECASE) and
       not re.search(patternList, key, re.IGNORECASE) and
       not re.search(patternTemplate, key, re.IGNORECASE) and
       not re.search(patternWikipedia, key, re.IGNORECASE) and
       not re.search(patternIndex, key, re.IGNORECASE) and
       not re.search(patternHtml, key, re.IGNORECASE) and
       not re.search(patternPercent, key) and
       not re.match(patternMillennium, key) and
       not val == "" and not val == " "):
      

      newDict.update({key: val})




  return newDict
